# Parler Data: Users with social media suspensions

The objective of this project is to explore the user data scraped from Parler after it's shutdown in early January 2021. The data was downloaded from this website (https://zenodo.org/record/4442460#.YEkHNpNKi9Y) which was made available to the public by researchers, https://arxiv.org/pdf/2101.03820.pdf. 

This notebook includes codes used to filter users to those who mention a suspension or ban from another social media platform in their bios. A CSV of these accounts was then exported for further analysis. Twitter API was used to determine the status of the Twitter accounts associated with the Parler usernames in this subset. The bios and follower counts of Parler usernames with corresponding active Twitter accounts were also pulled from Twitter API.

# Importing tools

In [39]:
import json
import pandas as pd
pd.set_option('display.max_columns', 1000)
import matplotlib
%matplotlib inline
from datetime import datetime
import numpy as np
import re

# Importing the data

Load the CSV of users, parler_users.csv, and prepare it for analysis. Convert the dates to a readable datetime format.

In [2]:
custom_date_parser = lambda x: None if pd.isnull(x) else datetime.strptime(str(x), "%Y%m%d%H%M%S")

In [3]:
df = pd.read_csv('parler_users.csv', 
                 parse_dates = ['joined'], 
                 date_parser=custom_date_parser,
                 dtype={'badges':str, 'bio':str, 'datatype':str, 'following':float,
                       'human':str, 'integration':str, 'media':float, 'muted':str, 
                      'posts':str, 'profilePhoto':str, 'verified':str})

/Users/sheridanwall/.pyenv/versions/3.8.2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,6,9,15,21,23,25,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = df.drop(columns=['Unnamed: 0'])

# Filtering data for previous suspensions

Create another subset of users that mention a ban or suspension on another platform. Data was manually verified to ensure accounts included in this subset were consistent with the goal of filtering in this manner.

In [12]:
def bio_search(regex_pattern):
    return df.loc[df["bio"].str.contains(regex_pattern, flags=re.IGNORECASE, regex=True, na=False)]

In [13]:
banned = bio_search('banned')
suspended = bio_search('suspend(ed)?')

/Users/sheridanwall/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [14]:
banned_users = pd.concat([banned, suspended])

In [16]:
banned_users.to_csv('banned_users.csv')

# Preparing data for Twitter API

Twitter API includes a character limit on usernames. Filter out the usernames that exceed the 16-character limit to streamline the Twitter API process.

In [40]:
banned_users = pd.read_csv('banned_users.csv')

In [32]:
banned_users['username'] = banned_users.username.astype(str)

In [33]:
banned_users['username_length'] = banned_users['username'].map(lambda x: len(x))

In [34]:
banned_short_users = banned_users[banned_users.username_length < 16]

In [22]:
banned_short_users.to_csv('banned_short_users.csv')

In [35]:
banned_short_users.shape

(3899, 32)

# Merging API results with original datasets

After creating a text file with the results of the Twitter API query, Save the text file into a CSV, and add column names: username, error_code, error_desc, error_details. Merge with the  filtered dataset used in the Twitter API analysis.

In [43]:
banned_results = pd.read_csv("banned_api_results.csv")

In [47]:
banned_results.head()

,username,error_code,error_desc,error_details
0,ronnieOD,200,account exists,NaN
1,poetWOAgun,200,account exists,NaN
2,Somaspecialist,200,account exists,NaN
3,DapiceDom,200,Not Found Error,Could not find user with usernames: [DapiceDom].
4,Rossicheryl,200,Forbidden,User has been suspended: [Rossicheryl].


In [53]:
banned_merged = banned_short_users.merge(banned_results, how='left', left_on='username', right_on='username')

In [58]:
banned_merged.username.value_counts(ascending=False)

Pumpkinhaid      1
RealJuanPablo    1
freeurmind8      1
deborahdiltz     1
KevinLaPage      1
                ..
Roadglide6       1
willibebanned    1
TGardIII         1
Guitar1norm      1
McVader          1
Name: username, Length: 3852, dtype: int64

In [61]:
banned_merged.shape

(3852, 36)

In [62]:
banned_results.shape

(3899, 4)

In [63]:
banned_short_users.shape

(3899, 33)

# Eliminating duplicates

Since the subset of suspended includes multiple keyword searches, if users had more than one of the keywords/phrases in their bios, than they were included in the data twice. Eliminate those users that were entered more than once. Export CSV and manually verify that each bio includes one of the groups mentioned above.

In [59]:
banned_merged = banned_merged.drop_duplicates(subset=['username'])

In [64]:
banned_merged.to_csv('banned_users_final.csv')

In [67]:
banned_merged.head()

,Unnamed: 0,Unnamed: 0.1,id,key,badges,banned,bio,blocked,comments,datatype,followed,user_followers,user_following,following,human,integration,isFollowingYou,joined,lastseents,likes,media,muted,pendingFollow,posts,private,profilePhoto,rss,state,userid,username,verified,verifiedComments,username_length,error_code,error_desc,error_details
0,0,160,1410466c39304af79f7f63cd8d4e2f6f,1410466c39304af79f7f63cd8d4e2f6f,[0],False,American #USN #Veteran. Overseas. Perma banned...,False,181.0,user_profile,False,547.0,1400.0,1400.0,True,False,False,2020-05-30 04:03:19,2020-12-20 07:17:02.286792 UTC,73.0,0.0,False,False,167.0,False,f769a374f67145e4ba82cf17687ac8c3,False,NaN,1410466c39304af79f7f63cd8d4e2f6f,ronnieOD,False,False,8,200,account exists,NaN
1,1,183,f7ba6f8f468e48bf8277ba3730557209,f7ba6f8f468e48bf8277ba3730557209,[],False,If U always TELL THE TRUTH you dont have to re...,False,13.0,user_profile,False,3900.0,3900.0,3900.0,False,False,False,2019-06-08 12:19:25,2020-12-22 02:28:58.88997 UTC,77.0,10.0,False,False,100.0,False,3a6c2032025248f7933e36a60e9c487e,False,NaN,f7ba6f8f468e48bf8277ba3730557209,poetWOAgun,False,False,10,200,account exists,NaN
2,2,992,ed194a7233b547fc89b32fb269ec6419,ed194a7233b547fc89b32fb269ec6419,[],False,"Another one of those crazy ""MAGA"" people. Happ...",False,180.0,user_profile,False,71.0,52.0,52.0,False,False,False,2020-08-16 01:08:22,2020-12-21 03:46:24.965232 UTC,438.0,0.0,False,False,429.0,False,c5f44d3b71a4436ca09929b234a6f311,False,2.0,ed194a7233b547fc89b32fb269ec6419,Somaspecialist,False,False,14,200,account exists,NaN
3,3,1130,125b3761ec284f09b755e38001dd15f8,125b3761ec284f09b755e38001dd15f8,[],False,"Family , Justice , Faith , Freedom. Antifa bus...",False,42.0,user_profile,False,114.0,374.0,374.0,False,False,False,2020-06-29 21:58:01,2021-01-09 22:53:11.339642 UTC,319.0,1.0,False,False,39.0,False,3b3e92c626ec4de0b1d2fb985ef8d9fa,False,NaN,125b3761ec284f09b755e38001dd15f8,DapiceDom,False,False,9,200,Not Found Error,Could not find user with usernames: [DapiceDom].
4,4,1433,0cb492d150fa457eacf739cfb748dc36,0cb492d150fa457eacf739cfb748dc36,[],False,***Banned from Twitter*** \nItalian Catholic A...,False,271.0,user_profile,False,1300.0,1300.0,1300.0,False,False,False,2019-08-01 12:18:55,2020-12-22 01:53:10.547645 UTC,7400.0,2.0,False,False,526.0,False,69f9cc03b3a44a7682b0c1aa79329b05,False,NaN,0cb492d150fa457eacf739cfb748dc36,Rossicheryl,False,False,11,200,Forbidden,User has been suspended: [Rossicheryl].


# Creating new dataframes for active and suspended accounts

Create two new dataframes: one for those usernames that have been suspended on Twitter and those whose usernames are still active on Twitter.

# Suspended accounts

In [68]:
suspended = banned_merged[banned_merged.error_desc == 'Forbidden']

To identify relevant examples for the story, I sorted the suspended dataframe by number of followers, largest to smallest.

In [70]:
suspended.sort_values(by='user_followers', ascending=False).head(25)

,Unnamed: 0,Unnamed: 0.1,id,key,badges,banned,bio,blocked,comments,datatype,followed,user_followers,user_following,following,human,integration,isFollowingYou,joined,lastseents,likes,media,muted,pendingFollow,posts,private,profilePhoto,rss,state,userid,username,verified,verifiedComments,username_length,error_code,error_desc,error_details
219,246,97048,eba75e525febff6f9e2161dc00bcda7e,eba75e525febff6f9e2161dc00bcda7e,"[1, 9, 2, 3, 0]",NaN,"Investigative journalist, CEO @IlloominateMedi...",False,NaN,user_profile,False,1500000.0,3100.0,NaN,True,True,NaN,2018-12-10 20:04:40,2020-12-23 14:54:27.051499 UTC,NaN,NaN,False,NaN,NaN,False,88c9a6da5d084e8baf3210ce74a24e07,True,NaN,eba75e525febff6f9e2161dc00bcda7e,LauraLoomer,True,False,11,200,Forbidden,User has been suspended: [LauraLoomer].
218,245,96844,99526563c1ad4c11a6378fca0d1552df,99526563c1ad4c11a6378fca0d1552df,[1],NaN,"Banned by ��Facebook, Twitter, Spotify, iHeart...",False,NaN,user_profile,False,336000.0,153.0,NaN,False,False,NaN,2019-05-10 18:40:18,2020-12-23 14:54:28.77576 UTC,NaN,NaN,False,NaN,NaN,False,937c6223ff634327997cb8a7e2112283,False,NaN,99526563c1ad4c11a6378fca0d1552df,InfoWars,True,False,8,200,Forbidden,User has been suspended: [InfoWars].
1409,1557,5933661,ac13e6a3e51e456bb1f1adccf52d0f25,ac13e6a3e51e456bb1f1adccf52d0f25,[0],False,"Mom, AF SMSgt (ret), Fmr Abn Rus Linguist, Top...",False,21.0,user_profile,False,75000.0,608.0,608.0,True,False,False,2020-03-29 15:11:46,2020-12-21 00:45:48.611434 UTC,348.0,158.0,False,False,826.0,False,0f4732d6b4034db7a9f090abcc3e288d,False,NaN,ac13e6a3e51e456bb1f1adccf52d0f25,Lisamei62,False,False,9,200,Forbidden,User has been suspended: [Lisamei62].
1519,1675,5989655,20ef027bed50432581d935c9b4e1e854,20ef027bed50432581d935c9b4e1e854,[0],NaN,SON OF THE NEW AMERICAN REVOLUTION #ALLLIVESMA...,False,NaN,user_profile,False,56000.0,87000.0,NaN,True,False,NaN,2019-06-04 22:45:47,2020-12-23 14:46:04.256064 UTC,NaN,NaN,False,NaN,NaN,False,04c462f12ebe4fbaa609e49cd619a688,False,NaN,20ef027bed50432581d935c9b4e1e854,freedommaker4u2,False,False,15,200,Forbidden,User has been suspended: [freedommaker4u2].
1031,1144,4446376,db60a7604ba64e348984907fb5ca9613,db60a7604ba64e348984907fb5ca9613,[1],False,The most banned videos & live content on the i...,False,15.0,user_profile,False,42000.0,100.0,100.0,False,False,False,2019-07-01 17:13:03,2020-12-20 16:38:38.966774 UTC,112.0,93.0,False,False,1800.0,False,a097fca72069403c8bc9b0bbb2d778d8,False,NaN,db60a7604ba64e348984907fb5ca9613,BannedDotVideo,True,False,14,200,Forbidden,User has been suspended: [BannedDotVideo].
346,387,1484113,2abc73d0445f43d1a40646b61a87d147,2abc73d0445f43d1a40646b61a87d147,[0],False,Banned by Twitter\n #MAGA ���� #liberalismisam...,False,1100.0,user_profile,False,32000.0,38000.0,38000.0,True,False,False,2019-05-29 02:48:54,2021-01-11 07:59:40.030697 UTC,1500.0,1000.0,False,False,3400.0,False,9e1f94ae76534a54b9df015532202123,False,NaN,2abc73d0445f43d1a40646b61a87d147,Lisaintexas,False,False,11,200,Forbidden,User has been suspended: [Lisaintexas].
3530,3867,5989750,ed5fb320fef142a8b01a749b96b774a5,ed5fb320fef142a8b01a749b96b774a5,[],NaN,Investor / founder / Tech enthusiast. #MAGA #M...,False,NaN,user_profile,False,31000.0,18000.0,NaN,False,False,NaN,2019-05-31 20:41:54,2020-12-23 14:46:05.664205 UTC,NaN,NaN,False,NaN,NaN,False,2ee399db81a94f75a286805bc3667e40,False,NaN,ed5fb320fef142a8b01a749b96b774a5,WinstonCovfefe,False,False,14,200,Forbidden,User has been suspended: [WinstonCovfefe].
94,110,31694,663bdf1dace54fc08b4a55399a47b43e,663bdf1dace54fc08b4a55399a47b43e,[0],False,MADE IN AMERICA - Banned on YouTube / Facebook...,False,16.0,user_profile,False,30000.0,151.0,151.0,True,False,False,2019-06-26 22:05:12,2020-12-21 04:06:23.720457 UTC,9.0,38.0,False,False,360.0,False,2796bb6c894d4edf8146c7f32168c8d2,False,NaN,663bdf1dace54fc08b4a55399a47b43e,allidoisowen,False,False,12,200,Forbidden,User has been suspended: [allidoisowen].
2957,3277,13077,0614b8f6bfec48f79e7785b2000c0a1a,0614b8f

In [71]:
suspended[suspended.user_followers >= 10000]

,Unnamed: 0,Unnamed: 0.1,id,key,badges,banned,bio,blocked,comments,datatype,followed,user_followers,user_following,following,human,integration,isFollowingYou,joined,lastseents,likes,media,muted,pendingFollow,posts,private,profilePhoto,rss,state,userid,username,verified,verifiedComments,username_length,error_code,error_desc,error_details
94,110,31694,663bdf1dace54fc08b4a55399a47b43e,663bdf1dace54fc08b4a55399a47b43e,[0],False,MADE IN AMERICA - Banned on YouTube / Facebook...,False,16.0,user_profile,False,30000.0,151.0,151.0,True,False,False,2019-06-26 22:05:12,2020-12-21 04:06:23.720457 UTC,9.0,38.0,False,False,360.0,False,2796bb6c894d4edf8146c7f32168c8d2,False,NaN,663bdf1dace54fc08b4a55399a47b43e,allidoisowen,False,False,12,200,Forbidden,User has been suspended: [allidoisowen].
218,245,96844,99526563c1ad4c11a6378fca0d1552df,99526563c1ad4c11a6378fca0d1552df,[1],NaN,"Banned by ��Facebook, Twitter, Spotify, iHeart...",False,NaN,user_profile,False,336000.0,153.0,NaN,False,False,NaN,2019-05-10 18:40:18,2020-12-23 14:54:28.77576 UTC,NaN,NaN,False,NaN,NaN,False,937c6223ff634327997cb8a7e2112283,False,NaN,99526563c1ad4c11a6378fca0d1552df,InfoWars,True,False,8,200,Forbidden,User has been suspended: [InfoWars].
219,246,97048,eba75e525febff6f9e2161dc00bcda7e,eba75e525febff6f9e2161dc00bcda7e,"[1, 9, 2, 3, 0]",NaN,"Investigative journalist, CEO @IlloominateMedi...",False,NaN,user_profile,False,1500000.0,3100.0,NaN,True,True,NaN,2018-12-10 20:04:40,2020-12-23 14:54:27.051499 UTC,NaN,NaN,False,NaN,NaN,False,88c9a6da5d084e8baf3210ce74a24e07,True,NaN,eba75e525febff6f9e2161dc00bcda7e,LauraLoomer,True,False,11,200,Forbidden,User has been suspended: [LauraLoomer].
346,387,1484113,2abc73d0445f43d1a40646b61a87d147,2abc73d0445f43d1a40646b61a87d147,[0],False,Banned by Twitter\n #MAGA ���� #liberalismisam...,False,1100.0,user_profile,False,32000.0,38000.0,38000.0,True,False,False,2019-05-29 02:48:54,2021-01-11 07:59:40.030697 UTC,1500.0,1000.0,False,False,3400.0,False,9e1f94ae76534a54b9df015532202123,False,NaN,2abc73d0445f43d1a40646b61a87d147,Lisaintexas,False,False,11,200,Forbidden,User has been suspended: [Lisaintexas].
381,424,1496003,86508c42f5464b4db12bb542b3afa225,86508c42f5464b4db12bb542b3afa225,[0],False,@POTUS @realDonaldTrump @FLOTUS Stage IV Throa...,False,2100.0,user_profile,False,17000.0,2200.0,2200.0,True,False,False,2019-05-30 15:33:57,2020-12-22 11:30:10.276675 UTC,14000.0,9400.0,False,False,71000.0,False,cd798a25276b4032bfe418148a8e688d,False,NaN,86508c42f5464b4db12bb542b3afa225,richirich1963,False,False,13,200,Forbidden,User has been suspended: [richirich1963].
395,438,1499835,ac645553c97c46dbb290debe1f852007,ac645553c97c46dbb290debe1f852007,[0],False,"#SuperBigMAGA ___Proud American, I am the larg...",False,534.0,user_profile,False,20000.0,23000.0,23000.0,True,False,False,2019-06-12 19:44:18,2020-12-22 03:51:16.340862 UTC,685.0,246.0,False,False,1500.0,False,3c5acab3fdee44ec8c9d37d9d74123df,False,NaN,ac645553c97c46dbb290debe1f852007,RalphRomano2,False,False,12,200,Forbidden,User has been suspended: [RalphRomano2].
418,462,1509948,71d54fc64545482da24b062f020b6ca7,71d54fc64545482da24b062f020b6ca7,[0],False,"Drummer,piano player, song writer, Conservativ...",False,502.0,user_profile,False,12000.0,18000.0,18000.0,True,False,False,2020-05-30 13:56:25,2020-12-20 10:59:50.086479 UTC,77.0,11.0,False,False,324.0,False,2ecb308b83ad452e95c29a6e83cdc23a,False,NaN,71d54fc64545482da24b062f020b6ca7,ShawnMalseed,False,False,12,200,Forbidden,User has been suspended: [ShawnMalseed].
519,580,1569553,05a34f5a220546489cc51bd31880a131,05a34f5a220546489cc51bd31880a131,"[1, 0]",NaN,"Journalist, Entrepreneur, and Father. Banned a...",False,NaN,user_profile,False,10000.0,381.0,NaN,True,False,NaN,2020-06-23 23:39:55,2020-12-23 14:46:05.456754 UTC,NaN,NaN,False,NaN,NaN,False,a4bc36bc8976448990b7aa7b48897965,False,NaN,05a34f5a220546489cc51bd31880a131,Edkrassen,True,False,9,200,Forbidden,User has been suspended: [Edkrassen].
716,793,2967084,7aefc6522fb943b0af31062

# Active accounts

In [72]:
active = banned_merged[banned_merged.error_desc == 'account exists']

In [73]:
active.shape

(1742, 36)

In [74]:
active.user_followers.sort_values(ascending=False).head(25)

221     1200000.0
2835     535000.0
856       60000.0
2164      32000.0
2834      32000.0
3295      31000.0
516       31000.0
2780      21000.0
1521      20000.0
2102      20000.0
1815      17000.0
1369      17000.0
518       17000.0
1056      16000.0
764       16000.0
3796      16000.0
2145      15000.0
722       15000.0
1376      15000.0
521       14000.0
520       13000.0
3389      13000.0
1520      13000.0
1424      12000.0
2023      11000.0
Name: user_followers, dtype: float64

In [75]:
active[active.user_followers >= 10000]

,Unnamed: 0,Unnamed: 0.1,id,key,badges,banned,bio,blocked,comments,datatype,followed,user_followers,user_following,following,human,integration,isFollowingYou,joined,lastseents,likes,media,muted,pendingFollow,posts,private,profilePhoto,rss,state,userid,username,verified,verifiedComments,username_length,error_code,error_desc,error_details
221,249,97318,39b8ada939a6482eb7b3ef1b477b4a12,39b8ada939a6482eb7b3ef1b477b4a12,[1],NaN,"Comedians, Conservatives ���� #MAGA #KAG \nRum...",False,NaN,user_profile,False,1200000.0,173.0,NaN,False,False,NaN,2019-06-12 05:37:17,2020-12-23 14:54:22.501705 UTC,NaN,NaN,False,NaN,NaN,False,b3a3e771bd5e4caaba2eb9f0e1932fb7,False,NaN,39b8ada939a6482eb7b3ef1b477b4a12,Hodgetwins,True,False,10,200,account exists,NaN
516,577,1568919,6585148b9835469b9bf4aa6de520e616,6585148b9835469b9bf4aa6de520e616,[0],NaN,Husband. Dog dad. ONU Pharmacy 2022. Twitch af...,False,NaN,user_profile,False,31000.0,25000.0,NaN,True,False,NaN,2020-06-24 12:47:31,2020-12-19 21:04:21.974382 UTC,NaN,NaN,False,NaN,NaN,False,dd8232cd918c41729874f909d505e051,False,NaN,6585148b9835469b9bf4aa6de520e616,DevinPierson,False,False,12,200,account exists,NaN
518,579,1569413,90b288d835eb4763ba72dd5d6988f44f,90b288d835eb4763ba72dd5d6988f44f,[],NaN,Banned on twitter\n\n�������� M-F 3pm-6pm CST:...,False,NaN,user_profile,False,17000.0,203.0,NaN,False,False,NaN,2020-06-25 19:14:08,2020-12-23 14:46:04.157763 UTC,NaN,NaN,False,NaN,NaN,False,ecb8aa1805d84ed18a58a628561da53c,False,NaN,90b288d835eb4763ba72dd5d6988f44f,Owenshroyer,False,False,11,200,account exists,NaN
520,581,1569664,f37ab7ab2f9c470b8f417f958d29d920,f37ab7ab2f9c470b8f417f958d29d920,[],NaN,"Father, Patriot, Veteran, Truth Warrior. Censo...",False,NaN,user_profile,False,13000.0,15000.0,NaN,False,False,NaN,2019-05-19 01:42:32,2020-12-23 14:46:03.295283 UTC,NaN,NaN,False,NaN,NaN,False,af4816ec135647199a1229d80a49ac88,False,NaN,f37ab7ab2f9c470b8f417f958d29d920,Dannmartin,False,False,10,200,account exists,NaN
521,582,1569681,3439ed8cd8ea482eb4704d5450d5fffe,3439ed8cd8ea482eb4704d5450d5fffe,[0],NaN,"Dad, US Marine, Hacker, Developer, Engineer, D...",False,NaN,user_profile,False,14000.0,30000.0,NaN,True,False,NaN,2020-10-08 15:54:12,2020-12-23 14:45:58.784769 UTC,NaN,NaN,False,NaN,NaN,False,f4ce2d3c978747629604febd5a274e63,False,1.0,3439ed8cd8ea482eb4704d5450d5fffe,JarradWinter,False,False,12,200,account exists,NaN
722,800,2968756,564c1a30fa0647a3a307bc33e5b452ff,564c1a30fa0647a3a307bc33e5b452ff,[],False,"Christian, Conservative, banned from Twitter, ...",False,164.0,user_profile,False,15000.0,22000.0,22000.0,False,False,False,2019-06-14 19:55:41,2020-12-22 00:39:36.465783 UTC,3700.0,62.0,False,False,7800.0,False,bb38c7a2cf8a4745afc777cb1fdcef81,False,NaN,564c1a30fa0647a3a307bc33e5b452ff,dpcollins,False,False,9,200,account exists,NaN
764,845,2984623,3d6a7b2350224502a238457c5af78bd5,3d6a7b2350224502a238457c5af78bd5,[0],False,"Conservative memes, jokes, and news. BANNED fr...",False,105.0,user_profile,False,16000.0,9200.0,9200.0,True,False,False,2019-08-11 21:00:48,2020-12-21 12:25:15.211218 UTC,6.0,903.0,False,False,2200.0,False,ac82564066e24439a61c799738808869,False,NaN,3d6a7b2350224502a238457c5af78bd5,realrightmemes,False,False,14,200,account exists,NaN
856,948,3040744,0f05587fd91e4b82a63b407e769870ba,0f05587fd91e4b82a63b407e769870ba,[],NaN,�������� M-F 3pm-6pm CST: https://banned.video...,False,NaN,user_profile,False,60000.0,99.0,NaN,False,False,NaN,2019-07-01 17:33:49,2020-12-23 14:54:38.492885 UTC,NaN,NaN,False,NaN,NaN,False,5aaa898c79384e94ae683397c644b183,False,NaN,0f05587fd91e4b82a63b407e769870ba,WarRoom,False,False,7,200,account exists,NaN
1056,1172,4455671,95dad3afae214d97af5b47993e81a54a,95dad3afae214d97af5b47993e81a54a,[],False,The voice of one of many shadow-banned Conserv...,False,13.0,user_profile,False,16000.0,17000.0,17000.0,False,False,False,2020-06-17 09:54:49,2020-12-22 04:02:46.61956 UTC,8.0,2100.0,False,False,4000.0,False,79780f10586248be8bad550c98542740,False,NaN,95dad3afae214

In [76]:
active.to_csv('banned_active_users.csv')

# Join Twitter API results to original active user dataframe

Create a CSV of the text file returned by the Twitter API search. Specify the following column names: username, error_code, bio, metrics, followers_count, following_count, tweet_count. Merge the CSV with the above active user dataframe. Export CSV for manual analysis.

In [ ]:
banned_active_api_results = pd.read_csv('banned_active_api_results.csv')

In [ ]:
banned_active_merged = active.merge(banned_active_api_results, how='left', left_on='username', right_on='username')

In [ ]:
banned_active_merged.to_csv('banned_active_twitter_info.csv')